# 시계열 관련 NumPy 및 Pandas 기능

## python, numpy, pandas 날짜 타입 비교 및 정리

|라이브러리|날짜, 시간 클래스| 타임델타 클래스|
|-------|-------------|-------------|
|datetime|datetime, date, time|timedelta|
|numpy|datetime64|timedelta64|
|panads|Timestamp|Timedelta|

- datetime은 python 설치 시 기본적으로 내장된 라이브러리로, 날짜를 쓸 것인지, 시간을 쓸 것인지, 날짜와 시간을 합쳐쓸 것인지에 따라 클래스가 분화되어 있음.   
예를들어 `2021-3-16` (날짜)을 표시하고 싶으면 `date` 클래스를 사용하고, `2021-3-16 12:34:2` (날짜+시간)를 표시하고 싶으면 `datetime` 클래스를 사용.  


- `datetime64`나 `Timestamp`는 각각 numpy, pandas 라이브러리에서 새로 정의한 날짜시간 클래스.   
이 둘은 한 클래스로 날짜, 시간, 날짜+시간을 모두 정의할 수 있는 것이 특징.

### 날짜 데이터 정의 - python 

In [1]:
import datetime

#날짜
print(datetime.date(2022, 5, 1))
#날짜 + 시간
print(datetime.datetime(2022, 5, 1, 15, 30, 45))
#시간
print(datetime.time(15, 30, 45))

2022-05-01
2022-05-01 15:30:45
15:30:45


### 날짜 데이터 정의 - numpy

In [2]:
import numpy as np 

#날짜
print(np.datetime64('2022-05-01'))

2022-05-01


### 날짜 데이터 정의 - pandas 
- pd.Timestamp :  python datetime.datetime 의 Pandas version
- pd.to_datetime :  Scalar, array-like, Series or DataFrame/dictionary 형태 data를 pandas DatetimeIndex로 날짜 변환.

In [3]:
import pandas as pd 

#날짜
print(pd.Timestamp(2022, 5, 1)) 

#날짜 + 시간
print(pd.Timestamp(year=2022, month=5, day=1, hour=12, minute=30)) 
print(pd.Timestamp('2022-05-01 15:30:30'))
print(type(pd.Timestamp('2022/05/01 15:30:30')))
print()

print(pd.to_datetime('2022-05-01 15:30:30'))
print(type(pd.to_datetime('2022/05/01 15:30:30')))
print(pd.to_datetime(['2022/05/01', '2022/05/02']))   # list

2022-05-01 00:00:00
2022-05-01 12:30:00
2022-05-01 15:30:30
<class 'pandas._libs.tslibs.timestamps.Timestamp'>

2022-05-01 15:30:30
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
DatetimeIndex(['2022-05-01', '2022-05-02'], dtype='datetime64[ns]', freq=None)


### Numpy 의 datetime64 format

- NumPy 날짜 배열은 ns(나노초) 단위의 datetime64 객체입니다. 
- 내부 저장 단위는 문자열 형태에서 자동으로 선택되며 날짜 단위 또는 시간 단위가 될 수 있습니다.   
- 날짜 단위는 년('Y'), 월('M'), 주('W'), 일('D')이고 시간 단위는 시('h'), 분('m'), 초('s'), 밀리초('ms') 입니다.

In [4]:
order_date = pd.Series(['2022-08-01 12:01:01', '2022-08-02 12:01:02', 
                                        '2022-08-03 12:01:03', '2022-08-04 12:01:04'])
order_date = pd.to_datetime(order_date)
order_date

0   2022-08-01 12:01:01
1   2022-08-02 12:01:02
2   2022-08-03 12:01:03
3   2022-08-04 12:01:04
dtype: datetime64[ns]

`datetime64[D]` - 날짜

In [5]:
order_date_daily = np.array(order_date, dtype='datetime64[D]')
order_date_daily

array(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04'],
      dtype='datetime64[D]')

`datetime64[M]` - 월

order_date_monthly = np.array(order_date, dtype='datetime64[M]')
order_date_monthly

In [6]:
np.unique(np.array(order_date, dtype='datetime64[M]'))

array(['2022-08'], dtype='datetime64[M]')

`'datetime64[Y]` - 연

In [7]:
np.unique(np.array(order_date, dtype='datetime64[Y]'))

array(['2022'], dtype='datetime64[Y]')

## DateTimeIndex 를 가진 Pandas 시계열 data 생성 및 처리

- Timestamp을 index 로 하는 data 를 시계열데이터 (TimeSeries) 라고 부른다. 즉, index 가 DatetimeIndex 인 데이터이다.

- 시계열관련 class 와 생성 방법

| class         |           설명          |                               생성방법 |Pandas Class|
|---------------|:-----------------------:|---------------------------------------:|------------:|
| Timestamp     |     하나의 timestamp    |                 to_datetime, Timestamp |pandas.Timestamp|
| DatetimeIndex | timestamp 타입의 인덱스 | to_datetime, date_range, DatetimeIndex |pandas.DatetimeIndex|
| Period        |       time period       |                                 Period |pandas.Period|

## to_datetime()

- 날짜/시간을 나타내는 **여러 종류의 문자열**을 자동으로 datetime 자료형으로 바꾼 후 DatetimeIndex 자료형 인덱스를 생성

In [26]:
date_str = ['2010-01-01', '2015, 7, 1', 'May, 1 2016', '2010/01/01',
                  'Dec, 25, 2019', 'DEC 1 2020', 'dec 20 2021', '2020 12 31',  '20201231' ]

pd.to_datetime(date_str)

DatetimeIndex(['2010-01-01', '2015-07-01', '2016-05-01', '2010-01-01',
               '2019-12-25', '2020-12-01', '2021-12-20', '2020-12-31',
               '2020-12-31'],
              dtype='datetime64[ns]', freq=None)

## date_range
```python
pd.date_range(start, end, periods, freq)
```

- 모든 날짜/시간을 일일히 입력할 필요없이 **시작일과 종료일** 또는 **시작일과 기간**을 입력하면 범위 내의 인덱스를 생성  

- freq
    - S: 초  
    - T: 분  
    - H: 시간  
    - D: 일(day)  
    - B: 주말이 아닌 평일 (Business Day)
    - W: 주(일요일)  
    - M: 각 달(month)의 마지막 날  
    - MS: 각 달의 첫날  

[frequency alias](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) 참조

start ~ 10일

In [9]:
pd.date_range(start='2016-9-1', periods=10)

DatetimeIndex(['2016-09-01', '2016-09-02', '2016-09-03', '2016-09-04',
               '2016-09-05', '2016-09-06', '2016-09-07', '2016-09-08',
               '2016-09-09', '2016-09-10'],
              dtype='datetime64[ns]', freq='D')

start~end 사이 평일

In [10]:
pd.date_range('2019-5-1', '2019-5-10', freq='B')

DatetimeIndex(['2019-05-01', '2019-05-02', '2019-05-03', '2019-05-06',
               '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10'],
              dtype='datetime64[ns]', freq='B')

start ~ 12 개월

In [11]:
dates = pd.date_range('2019-01-01', periods=12, freq='M')
dates

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
               '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31',
               '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', freq='M')

DataFrame의 index를 datetime으로 변경

In [12]:
a = np.random.standard_normal((12, 4))
df = pd.DataFrame(a, columns=['n1', 'n2', 'n3', 'n4'])
df.index = dates
df.head()

n1        n2        n3        n4
2019-01-31 -0.803803  0.659347 -0.184692  0.188075
2019-02-28 -0.121539 -1.258433 -0.103296  0.866630
2019-03-31  0.846509 -2.061791  0.384471 -0.561049
2019-04-30 -0.173008 -0.252409 -1.160188  1.039877
2019-05-31  0.431376  1.969334  1.687278 -0.210485

In [13]:
df.index

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
               '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31',
               '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', freq='M')

## Pandas DatetimeIndex를 이용한 작업

In [14]:
dates = pd.date_range('2020-08-01', '2021-08-01', freq='M')
a = np.random.randn(12)
df = pd.DataFrame({'date': dates, 'a': a})
df

date         a
0  2020-08-31 -0.367726
1  2020-09-30 -0.078976
2  2020-10-31 -0.066854
3  2020-11-30 -1.174497
4  2020-12-31 -0.988309
5  2021-01-31 -0.729615
6  2021-02-28 -1.135877
7  2021-03-31  0.483748
8  2021-04-30 -0.655689
9  2021-05-31  1.019392
10 2021-06-30  0.305527
11 2021-07-31 -0.900615

기존의  numpy datetime64 format 변수를 사용하여 인덱스 설정

In [15]:
df.set_index('date', inplace=True)
df

a
date                
2020-08-31 -0.367726
2020-09-30 -0.078976
2020-10-31 -0.066854
2020-11-30 -1.174497
2020-12-31 -0.988309
2021-01-31 -0.729615
2021-02-28 -1.135877
2021-03-31  0.483748
2021-04-30 -0.655689
2021-05-31  1.019392
2021-06-30  0.305527
2021-07-31 -0.900615

In [16]:
df.index

DatetimeIndex(['2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
               '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31'],
              dtype='datetime64[ns]', name='date', freq=None)

### Subsetting data

이제 DatetimeIndex를 사용하여 데이터 하위 집합을 선택할 수 있습니다.

In [17]:
df.loc['2020']

a
date                
2020-08-31 -0.367726
2020-09-30 -0.078976
2020-10-31 -0.066854
2020-11-30 -1.174497
2020-12-31 -0.988309

In [18]:
df['2020-08': '2020-12']

a
date                
2020-08-31 -0.367726
2020-09-30 -0.078976
2020-10-31 -0.066854
2020-11-30 -1.174497
2020-12-31 -0.988309

### Datetime Components

Pandas Datetime 변수에는 여러 가지 유용한 구성 요소가 있습니다. DatetimeIndex를 사용하여 월, 연도, 요일, 분기 등과 같은 항목을 추출할 수 있습니다.

DatetimeIndex에서 date 추출

In [19]:
df.index.day

Int64Index([31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31], dtype='int64', name='date')

- DatetimeIndex에서 day of week 추출 (Day of Week: Monday=0, Sunday=6)

In [20]:
df.index.dayofweek   

Int64Index([0, 2, 5, 0, 3, 6, 6, 2, 4, 0, 2, 5], dtype='int64', name='date')

In [21]:
df['DayofWeek'] = df.index.dayofweek
df.head()

a  DayofWeek
date                           
2020-08-31 -0.367726          0
2020-09-30 -0.078976          2
2020-10-31 -0.066854          5
2020-11-30 -1.174497          0
2020-12-31 -0.988309          3

## resample

- 시간 간격을 재조정   
- 원래의 데이터가 그룹으로 묶이기 때문에 그룹 연산을 해서 대표값을 구해야 한다.

In [22]:
ts = pd.Series(np.random.randn(100), 
               index=pd.date_range("2018-1-1", periods=100, freq="M"))
ts.head()

2018-01-31    1.443152
2018-02-28   -0.714541
2018-03-31   -0.362806
2018-04-30   -0.533262
2018-05-31   -1.268167
Freq: M, dtype: float64

주 단위로 down-sampling

In [23]:
ts.resample('W')

주 단위 평균

In [24]:
ts.resample('W').mean()  

2018-02-04    1.443152
2018-02-11         NaN
2018-02-18         NaN
2018-02-25         NaN
2018-03-04   -0.714541
                ...   
2026-04-05    0.184701
2026-04-12         NaN
2026-04-19         NaN
2026-04-26         NaN
2026-05-03   -0.648192
Freq: W-SUN, Length: 431, dtype: float64

월 단위 평균

In [25]:
ts.resample('M').mean() 

2018-01-31    1.443152
2018-02-28   -0.714541
2018-03-31   -0.362806
2018-04-30   -0.533262
2018-05-31   -1.268167
                ...   
2025-12-31    1.558427
2026-01-31   -0.021801
2026-02-28   -1.018816
2026-03-31    0.184701
2026-04-30   -0.648192
Freq: M, Length: 100, dtype: float64